In [2]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 50.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 73.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 42.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.41.3-py3-none-any.whl size=25316960 sha256=1a43776947cfb5e8744361a1ae910ef51354929a46d3bf7ba807a9a96de9b981
  Stored in directory: /root/.cache/pip/wheels/fc/76/11/5b953090eebf531f660948a30cd26e70260619f6480f186a5a
  Created wheel for keras_bert: filename=

In [2]:
from google.colab import drive
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ktrain
import joblib
import torch
from sklearn.model_selection import train_test_split
from ktrain import text
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.utils import resample
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding,BertTokenizer, BertForSequenceClassification

In [3]:
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/SatriaData/dataset_prediksi_cleaned.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv(file_path)
df

,Unnamed: 0,IDText,text
0,0,TXT0001,lu mau org2 prodemokrasi negara punya sempat b...
1,1,TXT0002,prabowo tanya soal hutang luar negeri jawab hu...
2,2,TXT0003,kiki daliyo ganjar pranowo beliau sosok mengag...
3,3,TXT0004,prabowo gibran laku semua sejahtera rakyat
4,4,TXT0005,justru nyambung junjung elu aomkmkmkmk ngomong...
...,...,...,...
995,995,TXT0996,bikin bangga deh ganjarmahfud mau alokasi teng...
996,996,TXT0997,pak jokowi pilpres 2024 besar hati rangkul pak...
997,997,TXT0998,sbaiknya got nga usah ikut debat dehnga jelas ...
998,998,TXT0999,biasa rembuk musyawarah gaya pimpin ganjar sej...


In [5]:
df_pred = df.drop(columns=['Unnamed: 0'])
df_pred.isna().sum()

IDText    0
text      0
dtype: int64

In [6]:
df_pred

,IDText,text
0,TXT0001,lu mau org2 prodemokrasi negara punya sempat b...
1,TXT0002,prabowo tanya soal hutang luar negeri jawab hu...
2,TXT0003,kiki daliyo ganjar pranowo beliau sosok mengag...
3,TXT0004,prabowo gibran laku semua sejahtera rakyat
4,TXT0005,justru nyambung junjung elu aomkmkmkmk ngomong...
...,...,...
995,TXT0996,bikin bangga deh ganjarmahfud mau alokasi teng...
996,TXT0997,pak jokowi pilpres 2024 besar hati rangkul pak...
997,TXT0998,sbaiknya got nga usah ikut debat dehnga jelas ...
998,TXT0999,biasa rembuk musyawarah gaya pimpin ganjar sej...


In [7]:
NBM_loaded = joblib.load('/content/drive/My Drive/SatriaData/NB_Multinomial_model.pkl')
knn_loaded = joblib.load('/content/drive/My Drive/SatriaData/knn_model.pkl')
vectorizer = joblib.load('/content/drive/My Drive/SatriaData/tfidf_vectorizer.pkl')

output_dir = '/content/drive/My Drive/SatriaData/IndoBERT_Model_Text_Classification'

tokenizer = BertTokenizer.from_pretrained(output_dir)
model = BertForSequenceClassification.from_pretrained(output_dir)

In [8]:
X_pred = vectorizer.transform(df_pred['text'])

df_pred['NB_Multinomial'] = NBM_loaded.predict(X_pred)
df_pred['KNN'] = knn_loaded.predict(X_pred)

inputs = tokenizer(df_pred['text'].tolist(), return_tensors="pt", padding=True, truncation=True, max_length=128)

with torch.no_grad():
    outputs = model(**inputs)
    indobert_predictions = torch.argmax(outputs.logits, dim=-1).numpy()
df_pred['IndoBERT'] = indobert_predictions
print(df_pred)

      IDText                                               text  \
0    TXT0001  lu mau org2 prodemokrasi negara punya sempat b...   
1    TXT0002  prabowo tanya soal hutang luar negeri jawab hu...   
2    TXT0003  kiki daliyo ganjar pranowo beliau sosok mengag...   
3    TXT0004         prabowo gibran laku semua sejahtera rakyat   
4    TXT0005  justru nyambung junjung elu aomkmkmkmk ngomong...   
..       ...                                                ...   
995  TXT0996  bikin bangga deh ganjarmahfud mau alokasi teng...   
996  TXT0997  pak jokowi pilpres 2024 besar hati rangkul pak...   
997  TXT0998  sbaiknya got nga usah ikut debat dehnga jelas ...   
998  TXT0999  biasa rembuk musyawarah gaya pimpin ganjar sej...   
999  TXT1000  mirage tolak juwono beli prabowo jubir timnas ...   

     NB_Multinomial  KNN  IndoBERT  
0                 1    1         1  
1                 1    1         1  
2                 4    4         4  
3                 1    1         1  
4         

In [10]:
df_pred['NB_Multinomial'] = df_pred['NB_Multinomial'].astype(int)
df_pred['KNN'] = df_pred['KNN'].astype(int)
df_pred['IndoBERT'] = df_pred['IndoBERT'].astype(int)

identical_predictions = df_pred[(df_pred['NB_Multinomial'] == df_pred['KNN']) &
                                      (df_pred['KNN'] == df_pred['IndoBERT'])]

print(f"Number of identical predictions across all models: {len(identical_predictions)}")

Number of identical predictions across all models: 655


In [11]:
different_predictions = df_pred[(df_pred['NB_Multinomial'] != df_pred['KNN']) |
                                      (df_pred['KNN'] != df_pred['IndoBERT']) |
                                      (df_pred['NB_Multinomial'] != df_pred['IndoBERT'])]

print(f"Number of different predictions across models: {len(different_predictions)}")

Number of different predictions across models: 345


In [12]:
df_pred.to_csv('/content/drive/My Drive/SatriaData/pred_final.csv')
